# Making Neural Network Model for Addition


## Data

In [1]:
import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy

Using TensorFlow backend.


In [2]:
def gen_data(num_examples, range_start, range_end):
    X = []
    y = []
    for ex in range(num_examples):
        num1 = int(random.random()*(range_end - range_start) + range_start)
        num2 = int(random.random()*(range_end - range_start) + range_start)

        X.append([num1,num2])        
        y.append(num1+num2)
    return np.array(X), np.array(y)

In [3]:
x_train, y_train = gen_data(10, -50, 50)
x_test, y_test = gen_data(10, 50, 200)

x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1,  x_test.shape[1])
[a.shape for a in [x_train, y_train, x_test, y_test]]

[(10, 1, 2), (10,), (10, 1, 2), (10,)]

## Create Models

### Model with 1 layer

In [4]:
model_1layer = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(1,2)),
    tf.keras.layers.Dense(1)
])

model_1layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy']
)

model_1layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 64)             192       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              65        
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________


### Model with 2 Layers

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, input_shape=(1,2)),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1, 128)            384       
_________________________________________________________________
dense_3 (Dense)              (None, 1, 64)             8256      
_________________________________________________________________
dense_4 (Dense)              (None, 1, 1)              65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


### Model 3 Layer


In [6]:
model_3layer = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, input_shape=(1,2)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model_3layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_3layer.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1, 256)            768       
_________________________________________________________________
dense_6 (Dense)              (None, 1, 128)            32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1, 64)             8256      
_________________________________________________________________
dense_8 (Dense)              (None, 1, 1)              65        
Total params: 41,985
Trainable params: 41,985
Non-trainable params: 0
_________________________________________________________________


### Model 4 Layer

In [7]:
model_4layer = tf.keras.models.Sequential([
  tf.keras.layers.Dense(512, input_shape=(1,2)),
  tf.keras.layers.Dense(256),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model_4layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_4layer.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1, 512)            1536      
_________________________________________________________________
dense_10 (Dense)             (None, 1, 256)            131328    
_________________________________________________________________
dense_11 (Dense)             (None, 1, 128)            32896     
_________________________________________________________________
dense_12 (Dense)             (None, 1, 64)             8256      
_________________________________________________________________
dense_13 (Dense)             (None, 1, 1)              65        
Total params: 174,081
Trainable params: 174,081
Non-trainable params: 0
_________________________________________________________________


### Model 5 Layer

In [8]:
model_5layer = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, input_shape=(1,2)),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
])
model_5layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_5layer.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 1, 1024)           3072      
_________________________________________________________________
dense_15 (Dense)             (None, 1, 512)            524800    
_________________________________________________________________
dense_16 (Dense)             (None, 1, 256)            131328    
_________________________________________________________________
dense_17 (Dense)             (None, 1, 128)            32896     
_________________________________________________________________
dense_18 (Dense)             (None, 1, 64)             8256      
_________________________________________________________________
dense_19 (Dense)             (None, 1, 1)              65        
Total params: 700,417
Trainable params: 700,417
Non-trainable params: 0
________________________________________________

## Train Model

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min')

### Train 1 Layer Model

In [10]:
history1 = model.fit(
    x_train, y_train,
    batch_size=10000,
    epochs=100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 251ms/step - loss: 1819.0084 - accuracy: 0.0000e+00 - val_loss: 23985.2598 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 45ms/step - loss: 750.0142 - accuracy: 0.0000e+00 - val_loss: 12595.6377 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 46ms/step - loss: 363.8660 - accuracy: 0.0000e+00 - val_loss: 6541.1724 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 46ms/step - loss: 178.9668 - accuracy: 0.0000e+00 - val_loss: 3256.2053 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 48ms/step - loss: 85.8029 - accuracy: 0.0000e+00 - val_loss: 1529.3542 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 50ms/step - loss: 39.2577 - accuracy: 0.0000e+00 - val_loss: 671.4337 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 42ms/step - loss: 16.8994 -

### Train 2 Layer Model

In [11]:
history2 = model.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 41ms/step - loss: 0.0129 - accuracy: 0.0000e+00 - val_loss: 0.0390 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 53ms/step - loss: 0.0410 - accuracy: 0.0000e+00 - val_loss: 1.0819 - val_accuracy: 0.0000e+00


### Train 3 Layer Model

In [12]:
history3 = model_3layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 316ms/step - loss: 1048.1448 - accuracy: 0.0000e+00 - val_loss: 807.5972 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 45ms/step - loss: 274.5759 - accuracy: 0.0000e+00 - val_loss: 776.7591 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 48ms/step - loss: 315.5811 - accuracy: 0.0000e+00 - val_loss: 172.9810 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 49ms/step - loss: 93.4987 - accuracy: 0.0000e+00 - val_loss: 122.8295 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 47ms/step - loss: 20.5657 - accuracy: 0.0000e+00 - val_loss: 10.9120 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 44ms/step - loss: 4.1633 - accuracy: 0.0000e+00 - val_loss: 5.6219 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 47ms/step - loss: 1.3302 - accuracy: 0.

### Train 4 Layer Model

In [13]:
history4 = model_4layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 238ms/step - loss: 1404.8245 - accuracy: 0.0000e+00 - val_loss: 28881.5664 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 45ms/step - loss: 2760.7715 - accuracy: 0.0000e+00 - val_loss: 10578.1973 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 46ms/step - loss: 963.9082 - accuracy: 0.0000e+00 - val_loss: 3009.7393 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 47ms/step - loss: 85.2351 - accuracy: 0.0000e+00 - val_loss: 70.8289 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 46ms/step - loss: 5.5540 - accuracy: 0.0000e+00 - val_loss: 4.7980 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 47ms/step - loss: 0.1676 - accuracy: 0.0000e+00 - val_loss: 0.0280 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 48ms/step - loss: 0.0717 - accuracy:

### Train 5 Layer Model

In [14]:
history5 = model_5layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 256ms/step - loss: 1393.6136 - accuracy: 0.0000e+00 - val_loss: 647275.6875 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 49ms/step - loss: 31736.7070 - accuracy: 0.0000e+00 - val_loss: 365662.0312 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 44ms/step - loss: 24070.9727 - accuracy: 0.0000e+00 - val_loss: 15758.3623 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 47ms/step - loss: 422.6984 - accuracy: 0.0000e+00 - val_loss: 5836.3018 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 46ms/step - loss: 149.9106 - accuracy: 0.0000e+00 - val_loss: 1773.3359 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 47ms/step - loss: 44.8566 - accuracy: 0.0000e+00 - val_loss: 406.4229 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 50ms/step - loss: 1

## Test Model

### Test 1 Layer Model

In [15]:
pred = model_1layer.predict(x_test)

In [16]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[ 60 106] expected_output=[166] prediction=[-4.273636] difference=[-170.27364] %-different from actual=[1.0257448]
input=[ 82 194] expected_output=[276] prediction=[-14.266139] difference=[-290.26614] %-different from actual=[1.0516889]
input=[53 92] expected_output=[145] prediction=[-3.494954] difference=[-148.49495] %-different from actual=[1.024103]
input=[118 172] expected_output=[290] prediction=[-2.1514225] difference=[-292.15143] %-different from actual=[1.0074188]
input=[192 111] expected_output=[303] prediction=[25.456663] difference=[-277.54333] %-different from actual=[0.91598463]
input=[101  69] expected_output=[170] prediction=[11.571947] difference=[-158.42805] %-different from actual=[0.9319297]
input=[123  50] expected_output=[173] prediction=[19.928068] difference=[-153.07193] %-different from actual=[0.88480884]
input=[ 90 145] expected_output=[235] prediction=[-4.009701] difference=[-239.0097] %-different from actual=[1.0170625]
input=[ 56 144] expected_output=

### Test 2 Layer Model

In [17]:
pred = model.predict(x_test)

In [18]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[ 60 106] expected_output=[166] prediction=[165.29663] difference=[-0.70336914] %-different from actual=[0.00423716]
input=[ 82 194] expected_output=[276] prediction=[274.2031] difference=[-1.7969055] %-different from actual=[0.00651053]
input=[53 92] expected_output=[145] prediction=[144.41641] difference=[-0.58358765] %-different from actual=[0.00402474]
input=[118 172] expected_output=[290] prediction=[289.11877] difference=[-0.8812256] %-different from actual=[0.00303871]
input=[192 111] expected_output=[303] prediction=[304.25806] difference=[1.2580566] %-different from actual=[0.004152]
input=[101  69] expected_output=[170] prediction=[170.53409] difference=[0.53408813] %-different from actual=[0.00314169]
input=[123  50] expected_output=[173] prediction=[174.18417] difference=[1.1841736] %-different from actual=[0.00684493]
input=[ 90 145] expected_output=[235] prediction=[234.12538] difference=[-0.87461853] %-different from actual=[0.00372178]
input=[ 56 144] expected_out

### Test 3 Layer Model

In [19]:
pred = model_3layer.predict(x_test)

In [20]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[ 60 106] expected_output=[166] prediction=[165.6014] difference=[-0.39860535] %-different from actual=[0.00240124]
input=[ 82 194] expected_output=[276] prediction=[275.40018] difference=[-0.599823] %-different from actual=[0.00217327]
input=[53 92] expected_output=[145] prediction=[144.66995] difference=[-0.3300476] %-different from actual=[0.00227619]
input=[118 172] expected_output=[290] prediction=[289.01703] difference=[-0.9829712] %-different from actual=[0.00338956]
input=[192 111] expected_output=[303] prediction=[301.2242] difference=[-1.7757874] %-different from actual=[0.00586068]
input=[101  69] expected_output=[170] prediction=[169.16113] difference=[-0.8388672] %-different from actual=[0.00493451]
input=[123  50] expected_output=[173] prediction=[171.92517] difference=[-1.0748291] %-different from actual=[0.00621289]
input=[ 90 145] expected_output=[235] prediction=[234.30072] difference=[-0.6992798] %-different from actual=[0.00297566]
input=[ 56 144] expected_out

### Test 4 Layer Model

In [21]:
pred = model_4layer.predict(x_test)

In [22]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[ 60 106] expected_output=[166] prediction=[165.62209] difference=[-0.37791443] %-different from actual=[0.00227659]
input=[ 82 194] expected_output=[276] prediction=[275.0674] difference=[-0.93258667] %-different from actual=[0.00337894]
input=[53 92] expected_output=[145] prediction=[144.70953] difference=[-0.2904663] %-different from actual=[0.00200322]
input=[118 172] expected_output=[290] prediction=[289.2153] difference=[-0.7846985] %-different from actual=[0.00270586]
input=[192 111] expected_output=[303] prediction=[302.61963] difference=[-0.3803711] %-different from actual=[0.00125535]
input=[101  69] expected_output=[170] prediction=[169.86647] difference=[-0.13352966] %-different from actual=[0.00078547]
input=[123  50] expected_output=[173] prediction=[172.9922] difference=[-0.00779724] %-different from actual=[4.507076e-05]
input=[ 90 145] expected_output=[235] prediction=[234.38074] difference=[-0.6192627] %-different from actual=[0.00263516]
input=[ 56 144] expecte

### Test 5 Layer Model

In [23]:
pred = model_5layer.predict(x_test)

In [24]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[ 60 106] expected_output=[166] prediction=[165.93723] difference=[-0.06277466] %-different from actual=[0.00037816]
input=[ 82 194] expected_output=[276] prediction=[275.81305] difference=[-0.18695068] %-different from actual=[0.00067736]
input=[53 92] expected_output=[145] prediction=[144.95438] difference=[-0.04562378] %-different from actual=[0.00031465]
input=[118 172] expected_output=[290] prediction=[289.87463] difference=[-0.12536621] %-different from actual=[0.0004323]
input=[192 111] expected_output=[303] prediction=[303.02637] difference=[0.02636719] %-different from actual=[8.7020424e-05]
input=[101  69] expected_output=[170] prediction=[170.02634] difference=[0.02633667] %-different from actual=[0.00015492]
input=[123  50] expected_output=[173] prediction=[173.07285] difference=[0.07284546] %-different from actual=[0.00042107]
input=[ 90 145] expected_output=[235] prediction=[234.8971] difference=[-0.10290527] %-different from actual=[0.00043789]
input=[ 56 144] expe